In [13]:
import pandas as pd
import re
import rapidfuzz as rf

# Citire Fisier si analiza coloane

In [17]:
df = pd.read_parquet("veridion_entity_resolution_challenge.snappy.parquet")

In [18]:
df.head(10)

,company_name,company_legal_names,company_commercial_names,main_country_code,main_country,main_region,main_city_district,main_city,main_postcode,main_street,...,generated_description,generated_business_tags,status,domains,all_domains,revenue,revenue_type,employee_count,employee_count_type,inbound_links_count
0,Owens Liquors,None,Owens Liquors,US,United States,South Carolina,None,Pawleys Island,29585,Ocean Highway,...,Owens Liquors is a retail establishment locate...,Retail Trade | Liquor Stores | Wine & Liquor,Active,None,None,None,None,None,None,None
1,Club Tarneit,None,Club Tarneit,AU,Australia,Victoria,Tarneit,City Of Wyndham,3029,None,...,None,None,Active,None,None,None,None,9,extracted,None
2,AAA Auto Otrokovice Zlín,None,AAA Auto Otrokovice Zlín,CZ,Czechia,Zlín,Kvítkovice U Otrokovic,Otrokovice,765 02,Zlínská,...,AAA Auto Otrokovice (Zlín) is a car dealership...,In-store Shopping | Investment Management Serv...,Active,None,None,None,None,None,None,None
3,Gisinger GmbH,Gisinger GmbH,None,DE,Germany,Baden-Württemberg,None,Ühlingen-Birkendorf,79777,Berauer Straße,...,None,None,Active,None,None,None,None,None,None,None
4,Kasana Life,None,Kasana Life,US,United States,Connecticut,None,Litchfield,06759,None,...,None,None,Active,None,None,None,None,None,None,None
5,BAM BROW & MAKEUP STUDIO,None,BAM BROW & MAKEUP STUDIO,AU,Australia,Western Australia,None,Mandurah,6201,None,...,None,None,Active,None,None,None,None,None,None,None
6,Tescoma,None,Tescoma,HU,Hungary,Budapest,None,Budapest,1222,Nagytétényi Út,...,Tescoma is a kitchen supply shop located in Bu...,Home Furnishings Retailer | Kitchenchen Supply...,Active,None,None,None,None,None,None,None
7,Happyweddings | No.1 Matrimony Trivandrum Kerala,None,Happyweddings | No.1 Matrimony Trivandrum Kera...,IN,India,Kerala,None,Thiruvananthapuram,695025,Medical College - Chalakkuzhy Road,...,Happyweddings.com is a company based in Thiruv...,Event Planning Services | Wedding Planning Ser...,Active,None,None,None,None,None,None,None
8,Dental Planet Manukau,None,Dental Planet Manukau,NZ,New Zealand,Auckland,None,Auckland,2104,Bakerfield Place,...,Dental Planet Manukau is a dental clinic locat...,Healthcare Services | Ddental Care Services,Active,None,None,None,None,None,None,None
9,Drake Design Photography,None,Drake Design Photography,US,United States,Texas,None,Lubbock,79416,County Road 6430,...,Drake Design Photography is a portrait studio ...,Wheelchair Access | Portrait Studio | Photogra...,Active,None,None,None,None,None,None,None


In [19]:
df.shape

(33446, 75)

In [20]:
df_entity.columns

NameError: name 'df_entity' is not defined

In [21]:
#sa vedem ce putem folosi ca si cheie
pd.set_option('display.max_rows', None)  

info_df = pd.DataFrame({
    "dtype": df.dtypes,
    "non_null_count": df.notna().sum(),
    "non_null_percent": df.notna().mean() * 100
}).sort_values(by="non_null_count", ascending=False)

print(info_df)

                               dtype  non_null_count  non_null_percent
status                        object           33446        100.000000
last_updated_at               object           33403         99.871435
created_at                    object           33403         99.871435
company_name                  object           32617         97.521378
website_domain                object           31893         95.356694
website_tld                   object           31893         95.356694
website_url                   object           31893         95.356694
main_country_code             object           31415         93.927525
main_country                  object           31415         93.927525
locations                     object           31415         93.927525
main_region                   object           30112         90.031693
main_city                     object           29602         88.506847
company_commercial_names      object           28121         84.078814
main_a

In [22]:
#sterg coloanele inutile pt ca doar imi ingreuneaza analiza au sub 2%
df.drop(columns=['naics_2022_secondary_codes','naics_2022_secondary_labels','android_app_url',
                 'ios_app_url','alexa_rank','tiktok_url','other_emails'],inplace=True)

In [23]:
#nu voi folosi adresa pentru ca este complicat de separat datele despre adresa si ma voi folosi de tara,oras,regine,postal_code
df['main_address_raw_text'].sample(30) 

8461                                     moscow, russia, -
15853    4200 Walnut Rd SE, Buckeye Lake, Ohio, 43008, ...
8770                Carr. Sevilla, 5A, 21007 Huelva, Spain
4918     79 Madison Ave, New York, New York, 10016, Uni...
9071                                   Edinburgh, Scotland
4712                      Unterm Ohmberg 1, 34431 Marsberg
22462    17, Soi Bearing 5, Bangna Tai, Bangna, Bangkok...
25516    NO. 26 Wucyuan 5th Road., New Taipei City, 248018
364                                                   None
32306             704 Rostraver Rd, Belle Vernon, PA 15012
9540           Driebergen-Rijsenburg, Utrecht, Netherlands
8465     1, Aravali Crescent, Nelson Mandela Marg, Vasa...
11098    Withycombe Village Rd, Exmouth EX8 3BD, United...
3533                      Rheingoldstraße 18, 10318 Berlin
8191                              🚛 London, United Kingdom
10389                                                 None
30977    House No # CEN(D)3, Flat: B1 Road no.95 Gulsha.

# Normalizare date

1. company_name

In [24]:
#se poate vedea ca sunt mai putine unice daca fac lower.
df['company_name'].str.lower().nunique(), df['company_name'].nunique()

(17273, 18715)

In [25]:
df['company_name'].sample(40)  

14301                          MarcasTV Registro de Marcas
26779                                    A-Met Engineering
274                                       Epworth Tandoori
7968                             Loami Village Center Hall
13804                                         Fresh Burger
29639                          KING DAVID ARCHITECTURE PC.
4390                                Alloy Wheel Repairs NI
1323                                             Cloud57uk
11513    Heritage Coffee Roasting Co. - Lemon Creek Dri...
32773                                               Aquido
2098                                      Jose's Courtroom
7617                                 General Cleaning Srl.
22524    SOUTHINGTON THE ATHLETIC SHOP LLC-SBA SMALL 7A...
8274                floristeria La Soledad de Almendralejo
7290                             Aquido Colon Hydrotherapy
26094              Mid Arabia Contracting Co.(Head Office)
6557                                                  No

In [26]:
stopwords = ['inc', 'llc', 'ltd', 'corp', 's.a.', 'srl', 'sa']

def normalize_company_name(name):
    if pd.isna(name):
        return None
    # lowercase + trim
    name = name.lower().strip()
    # eliminăm stopwords
    for word in stopwords:
        # cu spațiu înainte și după ca să nu taie părți din alte cuvinte
        name = re.sub(r'\b{}\b'.format(word), '', name)
    # eliminăm puncte, virgule, slash, multiple spații, minusuri
    name = re.sub(r'[-/&.,]', ' ', name)
    name = re.sub(r'\s+', ' ', name)  # un singur spațiu
    return name.strip()

In [27]:
#normalizez numele companiei
df['company_name_norm'] = df['company_name'].apply(normalize_company_name)

# verificăm un sample
df[['company_name', 'company_name_norm']].sample(20, random_state=42)

,company_name,company_name_norm
16837,Metalmaster Auto Body,metalmaster auto body
23910,The May Center for Learning,the may center for learning
8838,Break City,break city
17355,Amer Kebek,amer kebek
21660,Dipisa’s Pizza,dipisa’s pizza
11024,Dazzle Chocolate,dazzle chocolate
24947,Golden Life American Hospital,golden life american hospital
29217,Menjačnica Trange-Frange 24,menjačnica trange frange 24
19577,Katholische Jugend Volkmarsen,katholische jugend volkmarsen
17220,Dudek Manufaktura,dudek manufaktura


In [28]:
df['company_name_norm'].nunique()

16302

2. primary_phone

In [29]:
df['primary_phone'].sample(40) 

16164              None
29579              None
20499              None
7608       +12014881188
18765     +420735791661
17639      +85227959633
9498       +16516410878
8918      +393928879954
2594      +555532511555
27270              None
29403     +441208851511
29684      +79055341444
281      +4917733168152
20154      +77470940240
14627     +558432024402
9168      +441763246060
6834       +31251342332
13239      +19282770545
20095     +441916706864
932                None
20221              None
21719      +17705020226
22000    +4968168570101
2492       +81727602227
8522      +420800400450
28316              None
18752              None
29618    +4961367630019
2534       +16193337204
17371      +31612062275
3781      +573015017437
22039     +441277354795
32907      +17575664000
3473       +81568670314
13870              None
31748      +43722383274
7366      +966920022097
20723      +12067848330
26749              None
9883      +302103009323
Name: primary_phone, dtype: object

In [30]:
#scap de caractere non numerice
def normalize_phone(phone):
    if pd.isna(phone):
        return None
    return ''.join(filter(str.isdigit, str(phone)))

In [31]:
df['primary_phone_norm'] = df['primary_phone'].apply(normalize_phone)

In [32]:
df[['company_name_norm','primary_phone_norm','primary_phone']].sample(15)

,company_name_norm,primary_phone_norm,primary_phone
3566,monsieurhockey,15146600426,+15146600426
8252,maryport developments,441900814431,+441900814431
10094,modesto sunrise rotary club foundation,12095541850,+12095541850
18402,the italian job,None,None
16648,thurston screen printing embroidery,17075738717,+17075738717
2955,reedijk real estate,31152023095,+31152023095
21417,creative trails,12077545292,+12077545292
9969,madinum,None,None
9628,gästehaus niedermeierhof,4980941443,+4980941443
20901,clínica dental francisco lópez lópez,34968845214,+34968845214


3. website_domain


In [33]:
df['website_domain'].sample(30) 

9875       bistrolouisecatering.com
29186              riachuelo.com.br
16383            westbrookmaine.com
28833    ilginlerotomatikkepenk.com
23690                    capakt.com
24410       flowengineeringtech.com
28740                         wu.to
24294             copaiaestudio.com
7690             yorkpetsitters.com
17078                    wbce.co.uk
31374       karimullaindustries.com
28841      boats-cars-assistance.fr
22750          kimfitzgeralddmd.com
27285       reflectionsmirror.co.uk
16880                 grade92.co.uk
7692              binzel-abicor.com
2924       lucky7wineandliquors.com
2                        aaaauto.cz
2388                   posdcorb.com
8806                           None
6869                         amb.ma
33376                kochartech.com
30831          route12arcticcat.com
6486         galaxykitchendepot.com
3192         rivingtonspembroke.com
12500              fynehomes.org.uk
25056                  liberros.com
25449          volkswagenmad

In [34]:
df['website_domain_norm'] = df['website_domain'].str.lower().str.strip()

In [35]:
#nu a facut nicio diferenta, deci datele sunt bune in formatul in care sunt acum
df['website_domain_norm'].nunique(), df['website_domain'].nunique()

(6613, 6613)

4. main_country_code

In [36]:
df['main_country_code'].sample(30) 

20264    None
4517       DE
18738      BE
29786      DE
4288     None
12599    None
21827      CA
32143      DE
26169      US
1192       ES
4518       JP
30315      PT
30926      ES
4331       JP
22207      IT
25141      GB
20725      GB
17905    None
26990      DE
32448      US
15972      IN
30100      DE
17273    None
1172       FR
30321      RO
9293       FI
21849      US
8573       DE
25095      US
16978      US
Name: main_country_code, dtype: object

In [37]:
df['main_country_code_norm']=df['main_country_code'].str.lower()

In [38]:
df['main_country_code_norm'].nunique(),df['main_country_code'].nunique()

(137, 137)

5. main_region

In [39]:
df['main_region'].sample(30) 

31063                    None
13031              New Jersey
8185            North Holland
28398           Riyadh Region
21283                   Texas
19817        British Columbia
2595                    Idaho
13823                    Utah
16667              California
9603                 Michigan
31013           West Flanders
31826            Lower Saxony
28360                 England
20704           Massachusetts
5279               California
10761                 England
8425                  England
15086               Catalonia
15758    Auvergne-Rhône-Alpes
32208         New South Wales
20179                  Quebec
3195           Ústí Nad Labem
6886             Western Cape
19330                    Iowa
14686                 Antwerp
16250                  Oregon
19591                Virginia
27170                 Tuscany
55                    England
14967      Saitama Prefecture
Name: main_region, dtype: object

In [40]:
df['main_region_norm']=df['main_region'].str.lower().str.strip()

In [41]:
df['main_region_norm'].nunique(),df['main_region'].nunique()

(953, 953)

6. main_city

In [42]:
df['main_city'].sample(30) 

26601                   Ümraniye
10852                     Guadix
4544               South Croydon
371                         None
3114                Dar El Beïda
31938                     Moscow
20870                   Montreal
11134                  São Paulo
2039                     Preston
14051                  Carnaxide
10079                       None
8883                     Tallinn
6359               Santa Clarita
21520               Delray Beach
21475    Hawkesbury City Council
18116              West New York
20234              Andersonville
14853                 Alcobendas
12914                   Neuenhof
27738                     Tirana
32958                     Berlin
118                      Hausham
32598              Miami Springs
2294                      Irvine
8186                      Seneca
25121                       None
29785                   New York
10916                      Erith
22199                    Lincoln
10524                    Ashburn
Name: main

In [43]:
df['main_city_norm']=df['main_city'].str.lower().str.strip()

In [44]:
df['main_city_norm'].nunique(),df['main_city'].nunique()

(6673, 6673)

7. main_business_category

In [45]:
df['main_business_category'].sample(30) 

2686             Digital & Marketing Agencies
32402                                    None
3213          Clinics - Surgeons & Physicians
1562                          Animal Shelters
12267                Watches & Jewelry Stores
16002                                    None
24375                                    None
14045                                    None
15081                         Clothing Stores
18950                                    None
8103        Accounting & Bookkeeping Services
5967     Fruit & Vegetable - Markets & Stores
23800                                    None
32228                                    None
5623                                     None
6678                                     None
10153                           Art Galleries
7329                                     None
17321       Paints, Coatings, Dyes & Pigments
16238                         Clothing Stores
32193                                    None
18385                         Offi

In [46]:
def normalize_business_category(name):
    if pd.isna(name):
        return None
    # lowercase + trim
    name = name.lower().strip()
    # eliminăm puncte, virgule, slash, multiple spații, minusuri
    name = re.sub(r'[-/&.,]', ' ', name)
    name = re.sub(r'\s+', ' ', name)  # un singur spațiu
    return name.strip()

In [47]:
#normalizez numele companiei
df['main_business_category_norm'] = df['main_business_category'].apply(normalize_business_category)

# verificăm un sample
df[['main_business_category_norm', 'main_business_category']].sample(20, random_state=42)

,main_business_category_norm,main_business_category
16837,auto body shops,Auto Body Shops
23910,special needs education,Special Needs Education
8838,dance schools,Dance Schools
17355,None,None
21660,pizza italian restaurants,Pizza & Italian Restaurants
11024,None,None
24947,clinics surgeons physicians,Clinics - Surgeons & Physicians
29217,currency exchange service,Currency Exchange Service
19577,None,None
17220,paints coatings dyes pigments,"Paints, Coatings, Dyes & Pigments"


In [48]:
df['main_business_category_norm'].nunique(),df['main_business_category'].nunique()

(540, 540)

8. main_postcode

In [49]:
df['main_postcode'].sample(30) 

14151        07032
29962       625003
17485         7018
21043    83327-076
11049       394026
228           None
31532        19608
8723         76240
25004        22980
15677        30175
12175        72555
25802         7109
946          07960
20595         None
32086        22980
17322         None
27915      B92 0HF
18989       600088
31126     943-0804
22746         1274
29372        85204
10449         None
33252        47053
25451        S2300
20381       551 01
17530    12327-000
179          26003
33044         None
10258      PE9 2LD
5048          None
Name: main_postcode, dtype: object

In [50]:
def normalize_postcode(code):
    if pd.isna(code) or str(code).strip().lower() == "none":
        return None
    
    code = str(code).strip().upper()
    
    # elimin spațiile
    code = re.sub(r"\s+", "", code)
    
    # elimin -
    if re.match(r"^\d{3}-\d{4}$", code):
        return code.replace("-", "")
    
    # inlocuieste tot ce nu e litera mare sau cifra cu nimic
    code = re.sub(r"[^A-Z0-9]", "", code)
    
    return code

In [51]:
df["main_postcode_norm"] = df["main_postcode"].apply(normalize_postcode)

In [52]:
#am corectat cateva
df['main_postcode_norm'].nunique(),df['main_postcode'].nunique()

(10221, 10284)

# Deduplicare Date

1. Varianta cu toate coloanele in coloane principale

In [54]:
# Chei de grupare pentru deduplicare exactă
group_cols = ['company_name_norm', 'website_domain_norm', 'primary_phone_norm', 'main_city_norm', 'main_region_norm',
    'main_country_code_norm', 'main_postcode_norm']

df_dedup_1 =df.drop_duplicates(subset=group_cols, keep='first') \
             .reset_index(drop=True)

In [64]:
# Prag de similaritate
threshold = 90
company_names = df_dedup_1['company_name_norm'].tolist()
fuzzy_groups = {}
visited = set()

for i, name in enumerate(company_names):
    if name in visited:
        continue
    matches = rf.process.extract(
        name, company_names, scorer=rf.fuzz.token_sort_ratio, score_cutoff=threshold
    )
    group_ids = [company_names.index(match[0]) for match in matches]
    for idx in group_ids:
        visited.add(company_names[idx])
        fuzzy_groups[company_names[idx]] = name  # grupăm sub numele principal

df_dedup_1['company_name_fuzzy_group'] = df_dedup_1['company_name_norm'].map(fuzzy_groups)

# Deduplicare finală cu fuzzy group + domain
group_cols_fuzzy = ['company_name_fuzzy_group', 'website_domain_norm',]
df_final_1 = df_dedup_1.drop_duplicates(subset=group_cols_fuzzy, keep='first') \
                   .reset_index(drop=True)

print(f"Rânduri originale: {len(df)}")
print(f"Rânduri după deduplicare exactă: {len(df_dedup_1)}")
print(f"Rânduri după deduplicare fuzzy: {len(df_final_1)}")

Rânduri originale: 33446
Rânduri după deduplicare exactă: 27736
Rânduri după deduplicare fuzzy: 22411


2. Varianta cu 3 coloane principale si restul secundare si cu scor(in functie de cate din aceste coloane sunt completate, ramane randul cu cele mai multe din ele completate)

In [57]:
secondary_cols = [
    
    'main_city_norm', 'main_region_norm',
    'main_country_code_norm', 'main_postcode_norm',
    'main_business_category_norm'
]

def completeness_score(row):
    return sum([0 if pd.isna(row[col]) or row[col]=='' else 1 for col in secondary_cols])

df['completeness_score'] = df.apply(completeness_score, axis=1)

In [58]:
# Chei de grupare pentru deduplicare exactă
group_cols = ['company_name_norm', 'website_domain_norm', 'primary_phone_norm']

# Alegem rândul cu cel mai mare scor de completitudine
df_dedup_2 = df.sort_values('completeness_score', ascending=False) \
             .drop_duplicates(subset=group_cols, keep='first') \
             .reset_index(drop=True)

In [63]:
# Prag de similaritate
threshold = 90
company_names = df_dedup_2['company_name_norm'].tolist()
fuzzy_groups = {}
visited = set()

for i, name in enumerate(company_names):
    if name in visited:
        continue
    matches = rf.process.extract(
        name, company_names, scorer=rf.fuzz.token_sort_ratio, score_cutoff=threshold
    )
    group_ids = [company_names.index(match[0]) for match in matches]
    for idx in group_ids:
        visited.add(company_names[idx])
        fuzzy_groups[company_names[idx]] = name  # grupăm sub numele principal

df_dedup_2['company_name_fuzzy_group'] = df_dedup_2['company_name_norm'].map(fuzzy_groups)

# Deduplicare finală cu fuzzy group + domain
group_cols_fuzzy = ['company_name_fuzzy_group', 'website_domain_norm']
df_final_2 = df_dedup_2.sort_values('completeness_score', ascending=False) \
                   .drop_duplicates(subset=group_cols_fuzzy, keep='first') \
                   .reset_index(drop=True)

print(f"Rânduri originale: {len(df)}")
print(f"Rânduri după deduplicare exactă: {len(df_dedup_2)}")
print(f"Rânduri după deduplicare fuzzy: {len(df_final_2)}")

Rânduri originale: 33446
Rânduri după deduplicare exactă: 23930
Rânduri după deduplicare fuzzy: 17527
